In [8]:
import numpy as np
import litebird_sim as lbs

def add_lbsNoise(tod, det, block_duration_s):
   
    n_block = int(block_duration_s * det.sampling_rate_hz)
    n_samp = tod.shape[1]

    dets_random = [np.random.default_rng() for _ in range(tod.shape[0])]

    for start in range(0, n_samp, n_block):
        end = min(start + n_block, n_samp)
        
        lbs.add_noise(
            tod=tod[:, start:end],
            noise_type="one_over_f",
            sampling_rate_hz=det.sampling_rate_hz,
            net_ukrts=det.net_ukrts,
            fknee_mhz=det.fknee_mhz,
            fmin_hz=det.fmin_hz,
            alpha=det.alpha,
            dets_random=dets_random,
            scale=1.0,
        )

    return tod

    
    """
    Add 1/f noise to time-ordered data (TOD) for one or more detectors, in blocks.

    This function generates correlated 1/f noise that is continuous within each block
    and independent across different observations (TOD arrays). It supports TOD arrays
    of shape (D, N), where D is the number of detectors and N is the number of time samples.

    Parameters
    ----------
    tod : numpy.ndarray
        2D array of shape (D, N) containing the time-ordered data to which noise will be added.
        The array is modified in-place.
    det : litebird_sim.DetectorInfo
        Detector information object containing noise parameters:
        - sampling_rate_hz
        - fknee_mhz
        - fmin_hz
        - alpha
        - net_ukrts
        This implementation assumes all detectors share the same parameters.
    block_duration_s : float
        Duration of each processing block in seconds. The TOD is processed
        in blocks to optimize memory usage and performance.

    Returns
    -------
    numpy.ndarray
        The input `tod` array with added 1/f noise. Modifications are done in-place.
    """

In [13]:
import numpy as np
import ducc0

def add_OofaNoise(tod, det, block_duration_s):
    
    n_block = int(block_duration_s * det.sampling_rate_hz)
    n_samp = tod.shape[1]
   
    # Create one OofaNoise generator per detector
    noise_gens = [
        ducc0.misc.OofaNoise(
            sigmawhite=det.net_ukrts * 1e-6 * np.sqrt(det.sampling_rate_hz),
            f_knee=det.fknee_mhz * 1e-3,
            f_min=det.fmin_hz,
            f_samp=det.sampling_rate_hz,
            slope=-det.alpha,
        )
        for _ in range(tod.shape[0])
    ]

    # Iterate through the entire TOD in blocks of n_block samples
    for start in range(0, n_samp, n_block):    # block start index
        end = min(start + n_block, n_samp)     # block end index
        block_len = end - start                # effective block length

        # Generate independent noise per detector
        for d in range(tod.shape[0]):
            white_chunk = np.random.normal(0., 1., block_len)               # Generates white noise
            tod[d, start:end] += noise_gens[d].filterGaussian(white_chunk)  # Adds 1/f filter to TOD in place (vector view, not copy)

            del white_chunk # Frees memory explicitly

    return tod

    
    """
    Add 1/f (Oofa) noise to time-ordered data (TOD) for one or more detectors.

    This function generates **independent 1/f noise** for each detector and processes
    the TOD in blocks to optimize memory usage. The noise is applied **in-place**,
    modifying the original TOD array. It is compatible with TOD arrays of shape (D, N),
    where D is the number of detectors and N is the number of time samples.

    Parameters
    ----------
    tod : numpy.ndarray
        2D array of shape (D, N) containing the time-ordered data to which noise will be added.
        Modifications are done in-place.
    det : litebird_sim.DetectorInfo
        Detector information object containing noise parameters:
        - sampling_rate_hz
        - fknee_mhz
        - fmin_hz
        - alpha
        - net_ukrts
        This implementation assumes all detectors share the same parameters.
    block_duration_s : float
        Duration of each processing block in seconds. The TOD is processed in blocks
        to reduce memory usage for long arrays.

    Returns
    -------
    numpy.ndarray
        The same `tod` array with added 1/f noise (modified in-place).
    """